In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from qutip import about, basis, destroy, mesolve, qeye, sigmaz, tensor
%matplotlib inline
#iterakcija svetloba-snov Jaynes Cumming model. Posamezni dvonivojski atom iteragira z elektomagnetnim resonatorjem.
#mater enačbe rešuje mesolve
# H = \hbar \omega_{c} a^{\dagger} a + \frac{1}{2} \hbar \omega_{a} \sigma_{z} + \hbar g (a^{\dagger} + a)(\sigma_{-} + \sigma_{-})
#RWA -> H_{RWA} = \hbar \omega_{c} a^{\dagger} a + \frac{1}{2} \hbar \omega_{a} \sigma_{z} + \hbar g ( a^{\dagger} \sigma_{-} + a \sigma_{+})

N = 15  # number of cavity fock states
wc = 1.0 * 2 * np.pi  # cavity frequency
wa = 1.0 * 2 * np.pi  # atom frequency
g = 0.05 * 2 * np.pi  # coupling strength
kappa = 0.005  # cavity dissipation rate C_{1} = \sqrt{\kappa (1 + \langle n \rangle)} a , C_{2} = \sqrt{\kappa \langle n \rangle} a^{\dagger}
gamma = 0.05  # atom dissipation rate C_{3} = \sqrt{\gamma} \sigma_{-}
n_th_a = 0.0  # temperature in frequency units, \langle n \rangle = 0 nimamo kreacije fotonov, le anhilacijo pri tako izbranem stanju (vakumskem) 
use_rwa = True

tlist = np.linspace(0, 40, 500)

#setup operatorjev | stanje je tenzorski produkt cavty atom | cavity v osnovne stanju in atom v vzbujenem
psi0  = tensor(basis(N, 0), basis(2, 0))
# opeartorji kolapsa
a = tensor(destroy(N), qeye(2))
sm = tensor(qeye(N), destroy(2).dag()) #drugi del generira z destroy(2) sigma_{+}, ki ga z \dagger spravimo v anhilacijski operator
sz = tensor(qeye(N), sigmaz())

H = wc * a.dag() * a + 0.5 * wa * sz + g * (a.dag() + a) * (sm + sm.dag())
H_rwa = wc * a.dag() * a + 0.5 * wa * sz + g * (a.dag() * sm + a * sm.dag())

#za opis disipacije dodamo operatorje kolapsa 
c_operator_list = []
c_operator_list.append(np.sqrt(gamma) * sm) #anhilacija atoma 
c_operator_list.append(np.sqrt(kappa * ( 1 + n_th_a)) * a) # anhilacija fotona 
c_operator_list.append(np.sqrt(kappa * n_th_a) * a) #kreacija atoma

#razvoj sistema z disipacijo 
output = mesolve(H, psi0, tlist, c_operator_list, [a.dag() * a, sm.dag() * sm]) #razvijemo z Lindbladovo master enačbo in zahtevamo pričakovnae vrednosti a.dag a (cavity) in \sigma_{+} \sigma_{-} (atom)

"""
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(tlist, output.expect[0], label="Cavity")
ax.plot(tlist, output.expect[1], label="Atom excited state")
ax.legend()
ax.set_xlabel("Time")
ax.set_ylabel("Occupation probability")
ax.set_title("Vacuum Rabi oscillations at T={}".format(n_th_a));
"""